In [ ]:
!pip install transformers==4.41.2

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [12]:
device = "cuda"

In [2]:
text_fa = """
پیش‌بینی می‌شود اقتصاد فرانسه از یک دوره شش ماهه عدم رشد بهبود یابد و بانک مرکزی فرانسه رشد متوسط 0.1 تا 0.2 درصدی را در سه ماهه اول سال 2024 پیش‌بینی کرده است که ناشی از بهبود بخش خدمات و افزایش پیش‌بینی‌شده در بخش تولید صنعتی است.

به گفته فرانسوا ویلروی دو گالهاو، رئیس بانک مرکزی فرانسه،  فشارهای تورمی در حال کاهش یافتن است و شرکت‌های کمتری قیمت‌ها را افزایش می‌دهند و این امر منجر به کاهش تورم در ابعادی گسترده‌تر می‌شود که ممکن است بانک مرکزی اروپا را قادر سازد تا زمانی که تورم به طور مداوم در محدوده هدف ۲ درصد قرار گرفت، کاهش نرخ بهره را در نظر بگیرد.
"""


## Translate

In [74]:
translate_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
translate_tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [75]:
def translate(text, src_lang, tgt_lang):
  translate_pipe = pipeline("translation", model=translate_model, tokenizer=translate_tokenizer, src_lang=src_lang, tgt_lang=tgt_lang, device=device)
  translation = translate_pipe(text)
  return translation[0]['translation_text']

In [5]:
# translation = translate(text_fa, src_lang="fa_IR", tgt_lang="en_XX")

## Summarize

In [42]:
def summarize(text, model_name):
  summerize_tokenizer = AutoTokenizer.from_pretrained(model_name)
  summerize_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  summarize_pipe = pipeline("summarization", model=summerize_model, tokenizer=summerize_tokenizer , device=device)
  summary_max_length = len(summerize_tokenizer(text)['input_ids'])
  summary = summarize_pipe(text, min_length=1, max_length=summary_max_length)
  return summary[0]['summary_text']

In [9]:
text_to_summarize = translation

In [13]:
# summary = summarize(text_to_summarize, model_name="facebook/bart-large-cnn")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


## Compare

In [89]:
def summarize_by_translating(text):
  lang1 = "fa_IR"
  lang2 = "en_XX"
  translation = translate(text, src_lang=lang1, tgt_lang=lang2)
  summary = summarize(translation, model_name="facebook/bart-large-cnn")
  translated_summary = translate(summary, src_lang=lang2, tgt_lang=lang1)
  return translated_summary

In [90]:
def summarize_alone(text):
  summary = summarize(text, model_name="csebuetnlp/mT5_multilingual_XLSum")
  return summary

In [93]:
def compare(text):
  print("#"*100)
  print(f"input text length: {len(text)}")
  print(f"input text: {text}")
  print("#"*100)
  tst =  summarize_by_translating(text)
  print("#"*100)
  print(f"summarize_by_translating result length: {len(tst)}")
  print("summary by translation/summarization/translation:")
  print(tst,'\n')
  print("#"*100)
  s = summarize_alone(text)
  print("#"*100)
  print(f"summarize_alone result length: {len(s)}")
  print("summary by summarization alone:")
  print(s)
  print("#"*100)
  return tst,s

In [94]:
r1,r2 = compare(text_fa)

####################################################################################################
input text length: 563
input text: 
پیش‌بینی می‌شود اقتصاد فرانسه از یک دوره شش ماهه عدم رشد بهبود یابد و بانک مرکزی فرانسه رشد متوسط 0.1 تا 0.2 درصدی را در سه ماهه اول سال 2024 پیش‌بینی کرده است که ناشی از بهبود بخش خدمات و افزایش پیش‌بینی‌شده در بخش تولید صنعتی است.

به گفته فرانسوا ویلروی دو گالهاو، رئیس بانک مرکزی فرانسه،  فشارهای تورمی در حال کاهش یافتن است و شرکت‌های کمتری قیمت‌ها را افزایش می‌دهند و این امر منجر به کاهش تورم در ابعادی گسترده‌تر می‌شود که ممکن است بانک مرکزی اروپا را قادر سازد تا زمانی که تورم به طور مداوم در محدوده هدف ۲ درصد قرار گرفت، کاهش نرخ بهره را در نظر بگیرد.

####################################################################################################


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


####################################################################################################
summarize_by_translating result length: 145
summary by translation/summarization/translation:
اقتصاد فرانسه از یک دوره شش ماهه رکود بهبود پیدا می کند. بانک مرکزی فرانسه رشد متوسط 0.1 تا 0.2 درصد را در سه ماه اول سال ۲۰24 پیش بینی کرده است. 

####################################################################################################


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


####################################################################################################
summarize_alone result length: 90
summary by summarization alone:
بانک مرکزی فرانسه پیش بینی کرده است که اقتصاد این کشور در سه ماهه اول سال 2024 بهبود یابد.
####################################################################################################
